<h3> WIP NOTEBOOK CONTENTS </h3>

<img src = "https://i.imgur.com/EJj8QCN.png" ></img>

This is a simple Notebook to show Baseline Location in the world using the `folium` package.

<div id="toc_container" style="background: #f9f9f9; border: 1px solid #aaa; display: table; font-size: 95%;
                               margin-bottom: 1em; padding: 20px; width: auto;">
<p class="toc_title" style="font-weight: 700; text-align: center">Notebook Contents</p>
<ul class="toc_list">
  <li><a href="#baseline">1. Baseline Locations </a>
      <br>
      <ul>
    <li><a href="#baseline_geo">1.1 Geo Locations</a></li>
    <li><a href="#baseline_eda">1.2 Data Exploration</a></li>
  </ul>
</li>
</ul>
</div>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
root_path = '/kaggle/input/google-smartphone-decimeter-challenge'


<a id = "baseline">

In [ ]:
baseline_locations_train = pd.read_csv(root_path + "/baseline_locations_train.csv")
baseline_locations_test = pd.read_csv(root_path + "/baseline_locations_test.csv")
display(baseline_locations_train.sample(3))

<a id = 'baseline_eda'></a>

<a id = 'baseline_geo'></a>
<h5> Example of Locations: test (blue) vs train (orange) </h5>

In [ ]:
import folium

m = folium.Map(location=[37.453128,-122.154313], tiles='openstreetmap', zoom_start = 10)

sample_locations_test = baseline_locations_test.sample(200).reset_index(drop = True)
sample_locations_train = baseline_locations_train.sample(200).reset_index(drop = True)

for j in range(len(sample_locations_test)):
    try:
        folium.Marker(location=[sample_locations_test['latDeg'][j],
                                sample_locations_test['lngDeg'][j]],
                        popup=sample_locations_test['collectionName'][j],
                        icon = folium.Icon(prefix = 'fa', icon = "map-pin", color = 'lightblue'),
                        fill_color='#132b5e', num_sides=3, radius=3).add_to(m)
    except:
        continue
        
for j in range(len(sample_locations_train)):
    try:
        folium.Marker(location=[sample_locations_train['latDeg'][j],
                                sample_locations_train['lngDeg'][j]],
                        popup=sample_locations_train['collectionName'][j],
                        icon = folium.Icon(prefix = 'fa', icon = "map-pin", color = 'orange'),
                        fill_color='#132b5e', num_sides=3, radius=3).add_to(m)
    except:
        continue
m

<a id = "baseline"></a>
<h5> baseline_locations Exploration </h5>

In [ ]:
pn_train = (baseline_locations_train.groupby('phoneName').agg({'phone': ['nunique', 'count']})
            .reset_index(drop = False))
pn_train.columns = ['phoneName', 'unique_devices', 'n_rows']
pn_train = pn_train.assign(dataset='train')

pn_test = (baseline_locations_test.groupby('phoneName').agg({'phone': ['nunique', 'count']})
            .reset_index(drop = False))
pn_test.columns = ['phoneName', 'unique_devices', 'n_rows']
pn_test = pn_test.assign(dataset='test')


pn_dist = pd.concat([pn_train, pn_test], axis = 0, ignore_index = True)

cmap_plot = plt.get_cmap('jet_r')
plt.style.use('fivethirtyeight')

print("Number of unique devices per phoneName")
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(18, 6),gridspec_kw={'width_ratios': [1.5, 1.5]})
sns.barplot(x = 'phoneName', y = 'unique_devices', data = pn_dist, hue = 'dataset', ax = ax0)

ax0.set_xlim([-1, 7])
ax0.set_ylabel('Number of unique devices')
ax0.set_xticklabels(pn_dist.phoneName.unique().tolist(), rotation = 45)

font_size=12
bbox=[-0.2, 0, 1.2, 0.7]
ax1.axis('off')
ccolors = plt.cm.BuPu(np.full(len(pn_dist.columns), 0.1))
mpl_table = ax1.table(cellText = pn_dist.values, bbox=bbox, colLabels=pn_dist.columns, colColours=ccolors)
mpl_table.auto_set_font_size(True)
#mpl_table.set_fontsize(font_size)